In [67]:
from copy import deepcopy
import numpy as np
import matplotlib.pyplot as plt
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split

from src.models.estimator import RFClassifier, relu
from src.models.weights import V1_weights, classical_weights, sensilla_weights
from src.data.load_dataset import load_mnist, load_kmnist, generate_frequency_detection

In [62]:
# load dataset
train_loader, val_loader, test_loader = load_mnist()
train, test = train_loader.dataset.dataset, test_loader.dataset
train, train_labels = train.data.numpy(), train.targets.numpy()
X_test, y_test = test.data.numpy(), test.targets.numpy()

X_train, X_val, y_train, y_val = train_test_split(train, train_labels, train_size=0.85, stratify=train_labels)

In [63]:
# train params
h = 1000
bias = 0
nonlinearity = relu
scale = 1
clf = LinearSVC(tol=1e-4, max_iter=1000)

In [64]:
%%time
# define the V1 model
kwargs = {'size':5, 'spatial_freq': 2, 'center': None, 'scale': scale}
v1_clf = RFClassifier(h, V1_weights, bias, nonlinearity, deepcopy(clf), kwargs)
v1_clf.fit(X_train, y_train)
v1_clf.score(X_test, y_test)

[LibLinear]CPU times: user 1min 29s, sys: 332 ms, total: 1min 29s
Wall time: 1min 26s


/home/bpandey/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


0.9803

In [65]:
%%time
# define the classical model
kwargs = {'scale': scale}
classical_clf = RFClassifier(h, classical_weights, bias, nonlinearity, deepcopy(clf), kwargs)
classical_clf.fit(X_train, y_train)
classical_clf.score(X_test, y_test)

[LibLinear]CPU times: user 1min 18s, sys: 352 ms, total: 1min 18s
Wall time: 1min 15s


/home/bpandey/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


0.9493

### tseries classification

In [244]:
# train params
h = 250
bias = 0
nonlinearity = relu
scale = 1
clf = LinearSVC(tol=1e-4, max_iter=1000)

num_samples, sampling_rate, duration, freq, snr, seed = 7000, 1500, 0.1, 5, 0.8, 5
X, y = generate_frequency_detection(num_samples, sampling_rate, freq, duration, snr, seed)
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, stratify=y)

In [245]:
# define the sensilla model
kwargs = {'lowcut': 2, 'highcut': 8, 'decay_coef':36, 'scale': scale}
mech_clf = RFClassifier(h, sensilla_weights, bias, nonlinearity, deepcopy(clf), kwargs)
mech_clf.fit(X_train, y_train)
1 - mech_clf.score(X_test, y_test)

/home/bpandey/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


0.0007142857142856673

In [258]:
# define the classical model
kwargs = {'scale':scale}
classical_clf = RFClassifier(h, classical_weights, bias, nonlinearity, deepcopy(clf), kwargs)
classical_clf.fit(X_train, y_train)
1 - classical_clf.score(X_test, y_test)

/home/bpandey/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


0.010714285714285676